<a href="https://colab.research.google.com/github/alex-jk/SWB-GVCEH/blob/main/models/relevance_model/Relevance_Model_Reddit_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [8]:
file_path = '/content/drive/My Drive/SWB-GVCEH/VictoriaBC_data_updated.csv'
VictoriaBC_data_nodups = pd.read_csv(file_path)
VictoriaBC_data_nodups = VictoriaBC_data_nodups.drop_duplicates().reset_index(drop=True)

print("\nVictoria BC data ----------------")
print(VictoriaBC_data_nodups.shape)
print(VictoriaBC_data_nodups.columns)
print(VictoriaBC_data_nodups.head())

Sooke_data_nodups = pd.read_csv('/content/drive/My Drive/SWB-GVCEH/Sooke_data.csv')
Sooke_data_nodups = Sooke_data_nodups.drop_duplicates().reset_index(drop=True)

print("\nSooke data ----------------")
print(Sooke_data_nodups.shape)
print(Sooke_data_nodups.columns)
print(Sooke_data_nodups.head())


Victoria BC data ----------------
(8998, 6)
Index(['Subreddit', 'Title', 'Text', 'Submission ID', 'User ID',
       'Search Term'],
      dtype='object')
    Subreddit                                              Title  \
0  VictoriaBC  True change around homelessness from the homel...   
1  VictoriaBC  New transitional housing facility on Douglas o...   
2  VictoriaBC   Where to buy a reasonably priced Christmas tree?   
3  VictoriaBC  Looking for a Christmas tree? Please consider ...   
4  VictoriaBC                              Free clothing places?   

                                                Text Submission ID  \
0  I've started my own organization to expose the...        zen3ao   
1                                                NaN        5yah6z   
2  Went to the Christmas tree farm today and thou...       17yjdiw   
3                                                NaN        7h0e58   
4  Hey so long story short I moved here from Sask...       169ywlj   

          User 